In [1]:
import pandas as pd

In [2]:
weather_data_file = '../data/NOAA_weather_data/2005-2015_california_only_all.csv'
fire_data_file = '../data/USDA_wildfire_data/regridded_california_fires.csv'
training_data_file = '../data/training_data/2005-2015_training_data.csv'

# Parallelization options
N_PROCESSES = 15
JOBS_PER_PROCESS = 1

In [3]:
weather = pd.read_csv(weather_data_file)
weather['time'] = pd.to_datetime(weather['time'])
weather['date'] = weather['time'].dt.date
fires = pd.read_csv(fire_data_file)
fires['ignition'] = 1

FileNotFoundError: [Errno 2] File b'../data/USDA_wildfire_data/regridded_california_fires.csv' does not exist: b'../data/USDA_wildfire_data/regridded_california_fires.csv'

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11978560 entries, 0 to 11978559
Data columns (total 14 columns):
time        datetime64[ns]
lat         float64
lon         float64
air.sfc     float64
air.2m      float64
apcp        float64
crain       float64
rhum.2m     float64
dpt.2m      float64
pres.sfc    float64
uwnd.10m    float64
vwnd.10m    float64
veg         float64
date        object
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 1.2+ GB


In [5]:
fires.head()

,lat,lon,date,time,size,size_class,ignition
0,39.93427,-121.1598,2005-02-02,1300.0,0.10,A,1
1,38.85770,-120.5208,2004-05-12,845.0,0.25,A,1
2,39.13992,-120.5874,2004-05-31,1921.0,0.10,A,1
3,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1
4,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1


In [6]:
training_data = pd.merge(weather, fires, on=['lat', 'lon', 'date'], how='outer')

In [7]:
training_data.head()

,time_x,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,date,time_y,size,size_class,ignition
0,2005-01-01 00:00:00,40.29749,-124.3408,284.87308,283.24860,1.406250,1.0,77.472340,279.43463,98986.180,5.984521,3.811233,0.0,2005-01-01,NaN,NaN,NaN,NaN
1,2005-01-01 03:00:00,40.29749,-124.3408,284.71484,282.99774,0.781250,1.0,78.829380,279.48016,99001.150,3.470335,2.581940,0.0,2005-01-01,NaN,NaN,NaN,NaN
2,2005-01-01 06:00:00,40.29749,-124.3408,284.62878,282.55124,0.234375,1.0,77.277170,278.75513,99287.914,2.909558,1.104767,0.0,2005-01-01,NaN,NaN,NaN,NaN
3,2005-01-01 09:00:00,40.29749,-124.3408,284.64642,282.59882,1.875000,1.0,79.537100,279.22357,99256.030,2.127604,4.751480,0.0,2005-01-01,NaN,NaN,NaN,NaN
4,2005-01-01 12:00:00,40.29749,-124.3408,284.74457,282.18732,5.335938,1.0,80.818474,279.02075,99279.950,2.699432,6.547989,0.0,2005-01-01,NaN,NaN,NaN,NaN


In [8]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12168110 entries, 0 to 12168109
Data columns (total 18 columns):
time_x        datetime64[ns]
lat           float64
lon           float64
air.sfc       float64
air.2m        float64
apcp          float64
crain         float64
rhum.2m       float64
dpt.2m        float64
pres.sfc      float64
uwnd.10m      float64
vwnd.10m      float64
veg           float64
date          object
time_y        float64
size          float64
size_class    object
ignition      float64
dtypes: datetime64[ns](1), float64(15), object(2)
memory usage: 1.7+ GB


In [9]:
training_data.rename(columns={'time_x':'bin_time','time_y':'fire_discovery_time'}, inplace=True)
training_data.drop(['date'], axis=1, inplace=True)
training_data.head()

,bin_time,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,vwnd.10m,veg,fire_discovery_time,size,size_class,ignition
0,2005-01-01 00:00:00,40.29749,-124.3408,284.87308,283.24860,1.406250,1.0,77.472340,279.43463,98986.180,5.984521,3.811233,0.0,NaN,NaN,NaN,NaN
1,2005-01-01 03:00:00,40.29749,-124.3408,284.71484,282.99774,0.781250,1.0,78.829380,279.48016,99001.150,3.470335,2.581940,0.0,NaN,NaN,NaN,NaN
2,2005-01-01 06:00:00,40.29749,-124.3408,284.62878,282.55124,0.234375,1.0,77.277170,278.75513,99287.914,2.909558,1.104767,0.0,NaN,NaN,NaN,NaN
3,2005-01-01 09:00:00,40.29749,-124.3408,284.64642,282.59882,1.875000,1.0,79.537100,279.22357,99256.030,2.127604,4.751480,0.0,NaN,NaN,NaN,NaN
4,2005-01-01 12:00:00,40.29749,-124.3408,284.74457,282.18732,5.335938,1.0,80.818474,279.02075,99279.950,2.699432,6.547989,0.0,NaN,NaN,NaN,NaN


In [11]:
training_data.to_csv(training_data_file, index=False)